In [12]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.preprocessing import image
tf.keras.utils.to_categorical

<function tensorflow.python.keras.utils.np_utils.to_categorical(y, num_classes=None, dtype='float32')>

In [13]:
def predictPose(imageName, new_model):
    #resize the image since the model is trained with 150 by 150 images
    pose = image.load_img(imageName, target_size=(150,150))
    pose = image.img_to_array(pose)
    pose = np.expand_dims(pose, axis=0)
    pose = pose/255
    
    #actual classification
    prediction_prob = new_model.predict(pose)
    #returns which pose
    poseNumber = new_model.predict_classes(pose)
    return poseNumber[0]+1
    

In [14]:
def getSensitivitySpecifity(rootdir, char):
    path, dirs, files = next(os.walk(rootdir))
    action = 0
    actionList = []
    falsePositive = 0.0
    truePositive = 0.0
    falseNegative = 0.0
    trueNegative = 0.0
    catagory = 0
    
    if char == 'f':
        catagory = 1
    elif char == 'o':
        catagory = 2
    elif char == 's':
        catagory = 3
  
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filepath = subdir + os.sep + file
            if filepath.endswith(".jpg"):                
                #using the model to classify 
                action = predictPose(filepath)
                if action == catagory and file[0] == char: #true positive
                    print("classified", file, "as", action, ": true positive")
                    truePositive += 1
                elif action == catagory and file[0] != char: #false positive
                    print("classified", file, "as", action, ": false positive")
                    falsePositive += 1
                elif action != catagory and file[0] != char: #true negative
                    print("classified", file, "as", action, ": true negative")
                    trueNegative += 1
                elif action != catagory and file[0] == char: #fasle negative
                    print("classified", file, "as", action, ": false negative")
                    falseNegative += 1
                else:
                    print("ERROR: Unknown Classification")

    print("--------------\n")
    print("True Positive for " + char + " : " + str(truePositive))
    print("False Positive for " + char + " : " + str(falsePositive))
    print("True Negative for " + char + " : " + str(trueNegative))
    print("False Negative for " + char + " : " + str(falseNegative))

    sensitivity = truePositive/(truePositive+falseNegative)
    specifity = trueNegative/(falsePositive+trueNegative)
    accuracy = (truePositive+trueNegative)/(trueNegative+truePositive+falseNegative+falsePositive)

    print("\nSensitivity: ", sensitivity)
    print("Specificity: ", specifity)
    print("Accuracy: ", accuracy)

    actionList.append(action)

In [11]:
train_data = pd.read_csv('classification.csv', dtype=str)
#data = pd.read_csv('./Hand_Annotations_2.csv',dtype=str,names=colnames
Y = train_data[['label']]

kf = KFold(n_splits = 3)
                         
skf = StratifiedKFold(n_splits = 10, random_state = 7, shuffle = True)
image_shape = (150,150,3)

In [3]:
idg = ImageDataGenerator(#width_shift_range=0.1,
                         #height_shift_range=0.1,
                         #zoom_range=0.3,
                         fill_mode='nearest',
                         horizontal_flip = True)
                         #rescale=1./255)

In [4]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [5]:
#model = tf.keras.models.Sequential()
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128))
model.add(Activation('relu'))

# Dropouts help reduce overfitting by randomly turning neurons off during training.
# Here we say randomly turn off 50% of neurons.
model.add(Dropout(0.5))

# Last layer, not binary!
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy', #not binary for the poses
              optimizer='rmsprop',
              metrics=['accuracy'])

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
#predictions = model.predict_generator(train_data, 50)

In [16]:
VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

save_dir = 'kfoldModels/'
fold_var = 1

for train_index, val_index in kf.split(np.zeros(875),Y):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]

    train_data_generator = idg.flow_from_dataframe(training_data, target_size=image_shape[:2],
                               batch_size = 16,
                               x_col = "filename", y_col = "label",
                               class_mode = "categorical", shuffle = True)
    valid_data_generator  = idg.flow_from_dataframe(validation_data, target_size=image_shape[:2],
                            batch_size = 16,
                            x_col = "filename", y_col = "label",
                            class_mode = "categorical", shuffle = True)
    
  

    # CREATE NEW MODEL
    #model = create_new_model()

    # COMPILE NEW MODEL
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy',tf.keras.metrics.FalsePositives()])

    # CREATE CALLBACKS
    checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), 
                            monitor='val_accuracy', verbose=1, 
                            save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    # There can be other callbacks, but just showing one because it involves the model name
    # This saves the best model
    # FIT THE MODEL
    history = model.fit(train_data_generator,
                epochs=2,
                #steps_per_epoch=150,
                callbacks=callbacks_list,
                validation_data=valid_data_generator)
    
    
   
    
    #PLOT HISTORY

    # LOAD BEST MODEL to evaluate the performance of the model
    model.load_weights("kfoldModels/model_"+str(fold_var)+".h5")

    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names,results))

    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    
    #calculate sensitivity and specificity 
    

    tf.keras.backend.clear_session()

    fold_var += 1

    

Found 583 validated image filenames belonging to 3 classes.
Found 292 validated image filenames belonging to 3 classes.
Epoch 1/2
37/37 [==============================] - 8s 200ms/step - loss: 2.9923 - accuracy: 0.7804 - false_positives: 371.0000 - val_loss: 0.5068 - val_accuracy: 0.8630 - val_false_positives: 131.0000

Epoch 00001: val_accuracy improved from -inf to 0.86301, saving model to kfoldModels/model_1.h5
Epoch 2/2
37/37 [==============================] - 7s 203ms/step - loss: 0.3959 - accuracy: 0.9091 - false_positives: 327.0000 - val_loss: 0.8400 - val_accuracy: 0.8082 - val_false_positives: 177.0000

Epoch 00002: val_accuracy did not improve from 0.86301
19/19 [==============================] - 1s 37ms/step - loss: 0.4406 - accuracy: 0.8767 - false_positives: 133.0000
Found 583 validated image filenames belonging to 3 classes.
Found 292 validated image filenames belonging to 3 classes.
Epoch 1/2
37/37 [==============================] - 8s 201ms/step - loss: 1.3601 - accurac

**First 10-Fold Cross Validation Results:**

List of Accuracies of the 10 Models: [1.0, 0.98591548204422, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Average of 99.8591548204422%

*This doesn't include the images I used for testing*

In [9]:
VALIDATION_ACCURACY

[0.9090909361839294,
 1.0,
 0.9545454382896423,
 1.0,
 0.9886363744735718,
 1.0,
 1.0,
 1.0,
 0.9885057210922241,
 1.0]

In [10]:
(sum(VALIDATION_ACCURACY)/10) * 100

98.40778470039368